# 第五次课后练习 之二（选做）

在本次作业中，如果你在配置环境中遇到了问题，可以参考助教的解决方案：

首先我们在将这个文件放在一个文件夹中（可以和这门课程的其他作业放在一起，但是别放在桌面这种一大堆其他文件的文件夹里）。

随后运行：
```
python3 -m venv myenv
source myenv/bin/activate
pip install openai
pip install requests
pip install socksio
```

然后在 notebook 中切换 ipykernel 为 myenv（如果你采用的是 VSCode 的话，可以看看右上角的 ipykernel，点一下就可以切换了），就可以运行了。

如果你发现还存在包缺失的情况，注意采用 Restart，不然环境可能同步不过来。

**负责助教：吴迪**

<span style="color:red; font-weight:bold;">请将作业文件命名为 第五次课后练习-之二+姓名+学号.ipynb, 例如 第五次课后练习-之二+张三+1000000000.ipynb</span>

#  请认真阅读代码，理解学习代码的功能

## **0.1** 读取网页内容，调用大语言模型API进行中文摘要
    

In [ ]:
import re
import requests
from openai import OpenAI

# 目标URL列表
target_urls = [
    "https://arxiv.org/abs/2410.03761",
    "https://arxiv.org/abs/2305.15186"
]

# OpenRouter API配置，这个是提供免费deepseek服务的站点，
# 也可以替换成直接访问deepseekAPI的版本
API_ENDPOINT = "https://api.deepseek.com"
API_KEY = "sk-ee173942d8a748b7a9ff2e71f98e380e"

# 正则表达式，用于解析网页摘要（读取html网页对象的content字段内容）
pattern = r'<meta\s+property="og:description"\s+content="(.*?)"\s*/>' # content后面是得到摘要的内容

# 初始化OpenAI客户端
client = OpenAI(
    base_url=API_ENDPOINT,
    api_key=API_KEY,
)

def generate_summary(text: str) -> str:
    """生成文本摘要"""
    try:
        # 调用DeepSeek API生成摘要
        # 如果你采用了其他的站点的内容，你可能需要修改这里的 model 的内容。
        completion = client.chat.completions.create(
            model="deepseek/deepseek-chat:free",
            messages=[
                {
                    "role": "system",
                    "content": "Please write a 300 word(character) summary of the user's text in Chinese"
                },
                {
                    "role": "user",
                    "content": text
                }
            ]
        )
        # 返回生成的摘要内容
        return completion.choices[0].message.content
    except Exception as e:
        print(f"Error generating summary: {e}")
        return "Failed to generate summary"

def fetch_webpage_content(url: str) -> str:
    """获取网页内容"""
    try:
        # 发送HTTP GET请求获取网页内容
        response = requests.get(url, timeout=10)  # 设置超时时间
        response.raise_for_status()  # 检查请求是否成功
        return response.text
    except requests.exceptions.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return None

def parse_summary(html_content: str) -> str:
    """从网页内容中解析出摘要"""
    try:
        # 使用正则表达式从HTML中提取摘要，读取html网页对象的content字段
        match = re.search(pattern, html_content)
        if match:
            return match.group(1)
        else:
            raise Exception("No summary found in the webpage")
    except Exception as e:
        print(f"Error parsing summary: {e}")
        return None

def main():
    # 遍历目标URL列表
    
    docs = ""
    
    for url in target_urls:
        print(f"Processing URL: {url}")
        
        # 获取网页内容
        html_content = fetch_webpage_content(url)
        if not html_content:
            print('access error')
            continue  # 如果获取失败，跳过当前URL
        
        # 解析摘要
        summary = parse_summary(html_content)
        if not summary:
            print('parse error')
            continue  # 如果解析失败，跳过当前URL
            
        docs += summary
        
        # 生成中文摘要
        chinese_summary = generate_summary(summary)
        
        # 打印结果
        print(f"Original Summary: {summary}")
        print(f"Chinese Summary: {chinese_summary}\n")

    chinese_summary = generate_summary(docs)
    print(f"Original message: {docs}")
    print(f"multi doc Summary: {chinese_summary}\n")
if __name__ == "__main__":
    main()

我的输出：

Processing URL: https://arxiv.org/abs/2410.03761

Original Summary: Constructing taxonomies from citation graphs is essential for organizing scientific knowledge, facilitating literature reviews, and identifying emerging research trends. However, manual taxonomy construction is labor-intensive, time-consuming, and prone to human biases, often overlooking pivotal but less-cited papers. In this paper, to enable automatic hierarchical taxonomy generation from citation graphs, we propose HiGTL (Hierarchical Graph Taxonomy Learning), a novel end-to-end framework guided by human-provided instructions or preferred topics. Specifically, we propose a hierarchical citation graph clustering method that recursively groups related papers based on both textual content and citation structure, ensuring semantically meaningful and structurally coherent clusters. Additionally, we develop a novel taxonomy node verbalization strategy that iteratively generates central concepts for each cluster, leveraging a pre-trained large language model (LLM) to maintain semantic consistency across hierarchical levels. To further enhance performance, we design a joint optimization framework that fine-tunes both the clustering and concept generation modules, aligning structural accuracy with the quality of generated taxonomies. Extensive experiments demonstrate that HiGTL effectively produces coherent, high-quality taxonomies.

Chinese Summary: 本文提出了一种名为HiGTL（Hierarchical Graph Taxonomy Learning）的新型端到端框架，旨在从引用图中自动生成层次化分类体系，以解决手动构建分类体系时的工作量大、耗时长且易受人为偏见影响的问题。HiGTL通过结合文本内容和引用结构，递归地对相关论文进行层次化聚类，确保生成的簇在语义和结构上具有一致性。此外，该框架采用了一种新颖的分类节点生成策略，利用预训练的大型语言模型（LLM）迭代生成每个簇的核心概念，以保持层次间的语义一致性。为了进一步提升性能，HiGTL设计了一个联合优化框架，同时对聚类和概念生成模块进行微调，确保生成分类体系的结构准确性和质量。实验结果表明，HiGTL能够有效生成高质量且语义连贯的分类体系，为科学知识的组织和文献综述提供了有力支持。


Processing URL: https://arxiv.org/abs/2305.15186

Original Summary: Automatic literature review generation is one of the most challenging tasks in natural language processing. Although large language models have tackled literature review generation, the absence of large-scale datasets has been a stumbling block to the progress. We release SciReviewGen, consisting of over 10,000 literature reviews and 690,000 papers cited in the reviews. Based on the dataset, we evaluate recent transformer-based summarization models on the literature review generation task, including Fusion-in-Decoder extended for literature review generation. Human evaluation results show that some machine-generated summaries are comparable to human-written reviews, while revealing the challenges of automatic literature review generation such as hallucinations and a lack of detailed information. Our dataset and code are available at https://github.com/tetsu9923/SciReviewGen.

Chinese Summary: 自动文献综述生成是自然语言处理中最具挑战性的任务之一。尽管大型语言模型已经尝试解决这一问题，但缺乏大规模数据集一直是进展的障碍。我们发布了SciReviewGen数据集，包含超过10,000篇文献综述和69万篇被引用的论文。基于该数据集，我们评估了近期基于Transformer的摘要模型在文献综述生成任务中的表现，包括为文献综述生成扩展的Fusion-in-Decoder模型。人工评估结果显示，部分机器生成的摘要与人工撰写的综述相当，但也揭示了自动文献综述生成的挑战，如幻觉问题和缺乏详细信息。我们的数据集和代码可在https://github.com/tetsu9923/SciReviewGen获取。


Original message: Constructing taxonomies from citation graphs is essential for organizing scientific knowledge, facilitating literature reviews, and identifying emerging research trends. However, manual taxonomy construction is labor-intensive, time-consuming, and prone to human biases, often overlooking pivotal but less-cited papers. In this paper, to enable automatic hierarchical taxonomy generation from citation graphs, we propose HiGTL (Hierarchical Graph Taxonomy Learning), a novel end-to-end framework guided by human-provided instructions or preferred topics. Specifically, we propose a hierarchical citation graph clustering method that recursively groups related papers based on both textual content and citation structure, ensuring semantically meaningful and structurally coherent clusters. Additionally, we develop a novel taxonomy node verbalization strategy that iteratively generates central concepts for each cluster, leveraging a pre-trained large language model (LLM) to maintain semantic consistency across hierarchical levels. To further enhance performance, we design a joint optimization framework that fine-tunes both the clustering and concept generation modules, aligning structural accuracy with the quality of generated taxonomies. Extensive experiments demonstrate that HiGTL effectively produces coherent, high-quality taxonomies.Automatic literature review generation is one of the most challenging tasks in natural language processing. Although large language models have tackled literature review generation, the absence of large-scale datasets has been a stumbling block to the progress. We release SciReviewGen, consisting of over 10,000 literature reviews and 690,000 papers cited in the reviews. Based on the dataset, we evaluate recent transformer-based summarization models on the literature review generation task, including Fusion-in-Decoder extended for literature review generation. Human evaluation results show that some machine-generated summaries are comparable to human-written reviews, while revealing the challenges of automatic literature review generation such as hallucinations and a lack of detailed information. Our dataset and code are available at https://github.com/tetsu9923/SciReviewGen.

multi doc Summary: 本文提出了一种名为HiGTL（层次图分类学习）的新框架，旨在从引用图中自动生成层次化的科学文献分类体系。传统的分类体系构建方法依赖人工，耗时且易受偏见影响，可能忽略重要但引用较少的文献。HiGTL通过结合文本内容和引用结构，递归地对相关文献进行聚类，确保生成的分类在语义和结构上具有一致性。此外，该框架利用预训练的大语言模型（LLM）迭代生成每个聚类的核心概念，并通过联合优化框架进一步提升分类质量和结构准确性。实验表明，HiGTL能够生成高质量、连贯的分类体系。同时，本文还发布了SciReviewGen数据集，包含超过10,000篇文献综述和690,000篇被引文献，用于评估基于Transformer的文献综述生成模型。实验结果显示，部分机器生成的综述与人工撰写的质量相当，但也存在幻觉和细节不足等挑战。数据集和代码已公开。


# 1 仿照上面的框架，实现一个自己版本的多文档（或多轮对话）的分析功能。可以用更复杂的爬虫获得需要的信息。也可以直接从本地读取数据文件实现有实际意义的功能。

完成有新意有创意工作的同学，或在作业过程中觉得有心得或者自己拓展学习到有价值内容的，可以在文件名最后加一个#号。例如第五次课后练习+张三+1000000000+#.ipynb
只是完成学习的同学，没有尝试改进探索工作的可以不提交这个作业

2个代码（第一个成功而第二个未成功）

1.以下从本地文件读入，可以多轮对话。但是我的deepseek-api只有65536token，所以只能处理小文件。

In [ ]:
import os
import fitz
from openai import OpenAI

# OpenRouter API配置，这个是提供免费deepseek服务的站点，
# 也可以替换成直接访问deepseekAPI的版本
API_ENDPOINT = "https://api.deepseek.com"
API_KEY = "<API_KEY>"

# 初始化OpenAI客户端
client = OpenAI(
    base_url=API_ENDPOINT,
    api_key=API_KEY,
)


def generate_summary(text: str) -> str:
    """生成文本摘要"""
    try:
        # 调用DeepSeek API生成摘要
        completion = client.chat.completions.create(
            model="deepseek-chat",
            messages=[
                {
                    "role": "system",
                    "content": "Please write a 300 word(character) summary of the user's text in English"
                },
                {
                    "role": "user",
                    "content": text
                }
            ]
        )
        return completion.choices[0].message.content
    except Exception as e:
        print(f"Error generating summary: {e}")
        return "Failed to generate summary"


def read_local_file(file_path: str) -> str:
    """从本地文件读取内容"""
    if file_path.lower().endswith('.pdf'):
        return extract_text_from_pdf(file_path)
    else:
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                return file.read()
        except FileNotFoundError:
            print(f"File not found: {file_path}")
            return None
        except Exception as e:
            print(f"Error reading {file_path}: {e}")
            return None


def extract_text_from_pdf(pdf_path: str) -> str:
    """从PDF文件中提取文本"""
    try:
        document = fitz.open(pdf_path)
        text = ""
        for page_num in range(len(document)):
            page = document.load_page(page_num)
            text += page.get_text()
        return text
    except Exception as e:
        print(f"Error extracting text from PDF {pdf_path}: {e}")
        return None


def process_documents(documents: list) -> str:
    """处理多个文档并生成综合摘要"""
    combined_docs = ""

    for doc in documents:
        if os.path.exists(doc):
            content = read_local_file(doc)
            if content:
                combined_docs += content + "\n"
        else:
            print(f"Invalid path: {doc}")

    return combined_docs


def handle_multi_round_dialogue():
    """处理多轮对话"""
    docs = []
    while True:
        print("\nEnter a local file path (or type 'exit' to finish):")
        user_input = input()

        if user_input.lower() == 'exit':
            break

        if os.path.exists(user_input):
            content = read_local_file(user_input)
            if content:
                docs.append(content)
                english_summary = generate_summary(content)
                print(f"Original Document Content:\n{content}\n")
                print(f"English Summary: {english_summary}\n")
        else:
            print(f"Invalid path: {user_input}")

    if docs:
        combined_docs = "\n".join(docs)
        multi_doc_summary = generate_summary(combined_docs)
        print(f"Combined Documents Summary: {multi_doc_summary}\n")


def main():
    # 定义本地文件路径列表
    local_files = [
        "/Users/gtx/Desktop/Deep Learning (Ian Goodfellow, Yoshua Bengio, Aaron Courville).pdf",
        "/Users/gtx/Desktop/Real Analysis Stein.pdf"
    ]

    print("Starting document processing...")

    # 处理预定义的本地文件
    combined_docs = process_documents(local_files)

    if combined_docs:
        multi_doc_summary = generate_summary(combined_docs)
        print(f"Combined Documents Summary: {multi_doc_summary}\n")

    # 进入多轮对话模式
    handle_multi_round_dialogue()


if __name__ == "__main__":
    main()

2.以下是从bbc上爬取新闻，通过新闻标题判断新闻的立场是否支持中国，并统计出来。但是试了一些方法都没有成功爬取。后来又问了AI，改成用Selenium处理（因为部分内容是通过JavaScript动态加载的），但是也没有成功。

故这里放的是用静态HTML抓取。(改了多次仍未成功爬取)

In [ ]:
import re
import requests
from bs4 import BeautifulSoup

BASE_URL = "https://www.bbc.com/news/world/asia"
DEEPSEEK_API_ENDPOINT = "https://api.deepseek.com"
DEEPSEEK_API_KEY = "<API-KEY>"


def fetch_webpage_content(url: str, timeout: int = 30) -> str:
    """Fetch webpage content using requests with a specified timeout and User-Agent."""
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    try:
        response = requests.get(url, headers=headers, timeout=timeout)
        response.raise_for_status()
        return response.text
    except requests.exceptions.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return None


def parse_bbc_news(html_content: str):
    """Parse BBC news articles from HTML content."""
    soup = BeautifulSoup(html_content, 'html.parser')
    articles = []
    for h2 in soup.find_all('h2', {'data-testid': 'card-headline'}):
        link_tag = h2.find('a', href=True)
        if not link_tag:
            continue
        title = link_tag.get_text(strip=True)
        link = f"https://www.bbc.com{link_tag['href']}"
        articles.append((title, link))
    return articles


def clean_text(text: str) -> str:
    """Clean and preprocess text to avoid content risk issues."""
    # Remove special characters and excessive whitespace
    cleaned_text = re.sub(r'[^\w\s]', '', text)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    return cleaned_text


def analyze_sentiment(text: str, timeout: int = 30) -> str:
    """Analyze sentiment of text using DeepSeek deepseek-chat model with improved error handling and timeout."""
    cleaned_text = clean_text(text)
    headers = {
        "Authorization": f"Bearer {DEEPSEEK_API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": "deepseek-chat",
        "messages": [
            {
                "role": "system",
                "content": "请仔细阅读以下文本，并判断其立场是倾向于支持中国（pro-China）还是反对中国（anti-China）。仅返回 'pro-China' 或 'anti-China'。如果无法确定，则返回 'neutral'。"
            },
            {
                "role": "user",
                "content": cleaned_text
            }
        ]
    }
    try:
        response = requests.post(f"{DEEPSEEK_API_ENDPOINT}/chat/completions", json=payload, headers=headers,
                                 timeout=timeout)
        response.raise_for_status()
        completion = response.json()
        sentiment = completion['choices'][0]['message']['content'].strip().lower()
        if sentiment not in ["pro-china", "anti-china", "neutral"]:
            raise ValueError(f"Unexpected sentiment value: {sentiment}")
        return sentiment
    except Exception as e:
        print(f"Error analyzing sentiment: {e}")
        return "unknown"


def main():
    # Fetch the main page of BBC Asia
    html_content = fetch_webpage_content(BASE_URL)
    if not html_content:
        print("Failed to fetch BBC Asia homepage.")
        return

    # Print raw HTML content for debugging
    print("Raw HTML Content (first 1000 characters):")
    print(html_content[:1000])

    # Parse articles from the main page
    articles = parse_bbc_news(html_content)
    if not articles:
        print("No articles found.")
        return

    pro_china_count = 0
    anti_china_count = 0
    neutral_count = 0
    unknown_count = 0
    pro_china_titles = []
    anti_china_titles = []
    neutral_titles = []

    for title, link in articles:
        print(f"Processing article: {title}")

        sentiment = analyze_sentiment(title)
        if sentiment == "pro-china":
            pro_china_count += 1
            pro_china_titles.append(title)
        elif sentiment == "anti-china":
            anti_china_count += 1
            anti_china_titles.append(title)
        elif sentiment == "neutral":
            neutral_count += 1
            neutral_titles.append(title)
        else:
            unknown_count += 1

    # Output results
    print(f"Pro-China articles: {pro_china_count}")
    print(f"Anti-China articles: {anti_china_count}")
    print(f"Neutral articles: {neutral_count}")
    print(f"Unknown articles: {unknown_count}\n")
    print("Pro-China Titles:")
    for title in pro_china_titles:
        print(f"- {title}")
    print("\nAnti-China Titles:")
    for title in anti_china_titles:
        print(f"- {title}")
    print("\nNeutral Titles:")
    for title in neutral_titles:
        print(f"- {title}")


if __name__ == "__main__":
    main()



